In [67]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import argparse
import os 
import tqdm
parser = argparse.ArgumentParser(description='This is a demonstration program')
#parser.add_argument('-batch_size', type=str, required=True, help="Please provide a batch_size")

#args = parser.parse_args()

#print(f"batch_size:{args.batch_size}")
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)
block_size = 16
batch_size = 2
max_iters = 200
learning_rate = 3e-3
eval_iters=100
n_embd = 200
n_head = 4
dropout = 0.2
n_layer = 4

cuda


In [77]:
chars=""
with open('vocab.txt','r',encoding='utf-8') as f:
    text = f.read()
    chars = sorted(set(text))
vocab_size = len(chars)
print(chars)

['\x00', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x7f', '\x80', '\x81', '\x82', '\x83', '\x84', '\x88', '\x8c', '\x8d', '\x8f', '\x90', '\x91', '\x92', '\x93', '\x94', '\x95', '\x96', '\x97', '\x98', '\x99', '\x9c', '\x9d', '\x9f', '¡', '¢', '£', '¤', '¥', '¦', '§', '¨', '©', 'ª', '«', '¬', '\xad', '®', '¯', '°', '±', '²', '³', '´', 'µ', '¶', '·', '¸', '¹', 'º', '»', '¼', '½', '¾', '¿', 'À', 'Á', 'Â', 'Ã', 'Ä', 'Å', 'Æ', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ð', 'Ñ', 'Ò', 'Ó', 'Ô', 'Ö', '×', 'Ø', 'Ù', 'Ú', 'Û', 'Ü', 'Ý', 'Þ', 'ß', 'à', 'á', 'â', 'ã

In [14]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([ 0,  1,  1,  1,  2,  1,  3,  1,  4,  1,  5,  1,  6,  1,  7,  1,  8,  1,
         9,  1, 10,  1, 11,  1, 12,  1, 13,  1, 14,  1, 15,  1, 16,  1, 17,  1,
        18,  1, 19,  1, 20,  1, 21,  1, 22,  1, 23,  1, 24,  1, 25,  1, 26,  1,
        27,  1, 28,  1, 29,  1, 30,  1, 31,  1, 32,  1, 33,  1, 34,  1, 35,  1,
        36,  1, 37,  1, 38,  1, 39,  1, 40,  1, 41,  1, 42,  1, 43,  1, 44,  1,
        45,  1, 46,  1, 47,  1, 48,  1, 49,  1])


NameError: name 'files' is not defined

In [47]:

def get_random_chunk(split):
    filename ="output_train.txt" if split == 'train' else "output_val.txt"
    if not os.path.exists(filename) or os.path.getsize(filename) == 0:
        raise ValueError(f"The file '{filename}' does not exist or is empty.")
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
        # Determine the file size and a random position to start reading
            file_size=len(mm)
            start_pos = random.randint(0, file_size - (block_size * batch_size))
            #Seek to the random position and read the block of text 
            mm.seek(start_pos)
            block=mm.read(block_size*batch_size-1)
            #Decode the block to a string, ignoring any invalid byte sequences 
            decoded_block=block.decode('utf-8', errors='ignore').replace('\r', '')
            #Train and test splits
            data=torch.tensor(encode(decoded_block), dtype=torch.long)
    return data

def get_batch(split):
    data = get_random_chunk('train') if split == 'train' else get_random_chunk('val')
    ix = torch.randint(len(data) - block_size, (batch_size,))
    
    # Check for shape consistency
    x_list = [data[i:i+block_size] for i in ix]
    y_list = [data[i+1:i+block_size+1] for i in ix]
    
    assert all(x.shape[0] == block_size for x in x_list), "Mismatch in block size"
    assert all(y.shape[0] == block_size for y in y_list), "Mismatch in block size"
    
    x = torch.stack(x_list)
    y = torch.stack(y_list)

    # Print device assignment for x and y
    #print(f"x device: {x.device}, y device: {y.device}")

    x, y = x.to(device), y.to(device)
    return x, y

# Check the batch creation process
x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)


inputs:
tensor([[103, 111,  59,  32,  98, 117, 116,  32, 104, 101, 114,  32, 104, 117,
         115,  98],
        [100,  10, 116, 111, 108, 100,  32,  67, 104,  97, 114, 108, 101, 121,
          32, 104],
        [111, 105, 110, 103,  32, 115, 117,  99, 104,  32, 116, 104, 105, 110,
         103, 115],
        [108,  97, 110,  32,  97,  98, 111, 117, 116,  32,  79, 115,  99,  97,
         114,  32],
        [ 46,  34,  10,  10,  66, 121,  32,  97, 110, 100,  32,  98, 121,  32,
          77, 114],
        [ 97, 110, 105, 109,  97, 108, 115,  46,  34,  10,  10,  66, 121,  32,
          97, 110],
        [101,  32, 100, 111, 101, 115,  32, 115, 111,  46,  32,  72, 105, 115,
          32, 102],
        [ 97, 114, 108, 101, 121,  32,  98, 108, 117, 115, 104, 101, 100,  32,
          97, 110],
        [111, 100, 121,  32, 108, 111, 118, 101, 115,  32,  79, 115,  99,  97,
         114,  44],
        [114,  46,  10,  10,  83, 104, 101,  32, 101, 110,  99, 111, 117, 114,
          97, 103],
  

In [55]:
class Head(nn.Module):
    """ one head of self-attention """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.block_size = block_size
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        B, T, C = x.shape  # Batch size, Sequence length, Embedding size

        # Calculate Key, Query, Value matrices
        k = self.key(x)  # (B, T, head_size)
        q = self.query(x)  # (B, T, head_size)
        v = self.value(x)  # (B, T, head_size)
        # Self-attention scores (scaled dot-product attention)
        wei = q @ k.transpose(-2, -1) / (k.shape[-1] ** 0.5)  # (B, T, T)
        # Apply mask to ensure causal structure (look only at past tokens)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))

        # Softmax to get attention weights
        wei = F.softmax(wei, dim=-1)
        # Dropout on the attention weights (for regularization)
        wei = self.dropout(wei)
        # Weighted sum of value vectors
        out = wei @ v  # (B, T, head_size)
        return out
        
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel  """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) #(B,T,F) -> (B,T ,[h1,h1,h1,h1,h2,h2,h2,h2,h3,h3,h3,h3])
        out = self.dropout(self.proj(out))
        return out
        
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    def __init__(self,n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 + n_embd),
            nn.ReLU(),
            nn.Linear(4 + n_embd,n_embd),
            nn.Dropout(dropout),
        )
    def forward(self,x):
        return self.net(x)

class Block(nn.Module):
    """Transformer block: communication followed by computation"""
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd//n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x+y)
        y = self.ffwd(x)
        x = self.ln2(x+y)
        return x
         
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size,n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def _init_weights(self):
        """Custom weight initialization for GPT model"""
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)  # Xavier initialization for Linear layers
                if module.bias is not None:
                    nn.init.zeros_(module.bias)  # Zero-initialize biases
            elif isinstance(module, nn.Embedding):
                nn.init.normal_(module.weight, mean=0.0, std=0.02)  # Normal initialization for embeddings
            elif isinstance(module, nn.LayerNorm):
                nn.init.ones_(module.weight)  # Initialize weight for LayerNorm
                nn.init.zeros_(module.bias)   # Initialize bias for LayerNorm

    def forward(self, index, targets):
        B,T = index.shape
        tok_emb = self.token_embedding_table(index) #(B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))#(B, T, C)
        x = tok_emb + pos_emb#(B, T, C)
        x = self.blocks(x)#(B, T, C)
        x = self.ln_f(x)#(B, T, C)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits,loss
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index,None)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index,index_next),dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)

In [57]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()  # Set model to evaluation mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            
            # Pass both X (inputs) and Y (targets) to the model
            logits, loss = model(X, Y)
            
            losses[k] = loss.item()  # Store loss for this batch
            
        out[split] = losses.mean().item()  # Mean loss for this split
    model.train()  # Return model to training mode
    return out


In [63]:

with open('model-01.pkl','wb') as f:
    pickle.dump(model, f)
print('model saved')


model saved


In [61]:
 # pytorch optimizer
model = GPTLanguageModel(vocab_size)  # Initialize your model
model = model.to(device)  # Move the model to the correct device (CPU or GPU)

optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)

for iter in range(max_iters):
    # sample a batch of data
    if iter  % eval_iters==0:
        losses = estimate_loss()
        print(f'step:{iter}, train_loss:{losses['train']:.4f}, val_loss:{losses['val']:.4f}')
    xb, yb = get_batch('train')
    #evaluate the loss
    logits, loss = model.forward(xb,yb)
    
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

with open('model-01.pkl','wb') as f:
    pickle.dump(model, f)
print('model saved')

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
prompt = 'Hello! Can you see me?'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)